<a href="https://colab.research.google.com/github/Gabriela2685/Simulacion1/blob/main/Inventario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random as rnd
import math
import pandas as pd

In [ ]:
def asignar_valor(R, intervalos):           # Definimos una funcion que asigne el numero de demanda para cada mes
  for inicio, fin, valor in intervalos:     # en los intervalos que despues se proponen, a traves de un num aleatorio R
    if inicio <= R < fin:
      return valor
  return None

In [ ]:
def tiempo_entrega(R, T_entrega):           # Definimos una funcion que asigne el tiempo que va a tardar cada orden
  for inicio, fin, valor in T_entrega:      # A traves de un numero aleatorio R
    if inicio <= R < fin:
      return valor
  return None

In [ ]:
def Inv_entario():                          # Definimos nuestra funcion de inventario
    valores_meses = [                                  # Factor estacional que le corresponde a cada mes
        (1, 1.2), (2, 1.00), (3, 0.90), (4, 0.80),
        (5, 0.80), (6, 0.70), (7, 0.80), (8, 0.90),
        (9, 1.00), (10, 1.20), (11, 1.30), (12, 1.40)
    ]

    intervalos = [                                     # Dependiendo de donde caiga mi R se le asigna un valor a x
        (0.000, 0.010, 35), (0.010, 0.025, 36), (0.025, 0.045, 37),
        (0.045, 0.065, 38), (0.065, 0.087, 39), (0.087, 0.110, 40),
        (0.110, 0.135, 41), (0.135, 0.162, 42), (0.162, 0.190, 43),
        (0.190, 0.219, 44), (0.219, 0.254, 45), (0.254, 0.299, 46),
        (0.299, 0.359, 47), (0.359, 0.424, 48), (0.424, 0.494, 49),
        (0.494, 0.574, 50), (0.574, 0.649, 51), (0.649, 0.719, 52),
        (0.719, 0.784, 53), (0.784, 0.844, 54), (0.844, 0.894, 55),
        (0.894, 0.934, 56), (0.934, 0.964, 57), (0.964, 0.980, 58),
        (0.980, 0.995, 59), (0.995, 1.000, 60),
    ]

    T_entrega = [                                       # Dependiendo de donde caiga R se aigna un tiempo de llegada a la orden
        (0.00, 0.30, 1), (0.30, 0.70, 2), (0.70, 1.00, 3),
    ]

    Inv_I = 150               # Inventario inicial
    Ordenes = 0               # Contador para las ordenes que se generen
    Pedido_pendiente = 0      # Contador para los pedidos pendientes
    Mes_pedido = {}
    Faltante = 0              # Contador para los faltantes

    Mes = []
    Inv_inicial = []
    Num_aleatorio = []
    Dem_ajustada = []
    Inv_final = []
    Faltantes = []
    Num_orden = []
    Inv_promedio = []

    # Usamos un bucle for para trabajar del mes 1 al 12
    for mes, valor in valores_meses:
        R = rnd.random()                      # Generamos un valor aleatorio R entre 0 y 1
        x = asignar_valor(R, intervalos)      # Asignamos el valor de la demanda
        DAj = valor * x                       # Ajuste de la demanda (Se aplica el factor estacional)
        Dem_ajust = math.floor(DAj)           # Redondeamos

        # Guardamos el inventario inicial antes de actualizarlo
        Inv_inicial.append(Inv_I)

        if mes in Mes_pedido:
            Inv_I += Mes_pedido[mes]
            Pedido_pendiente -= Mes_pedido[mes]
            del Mes_pedido[mes]

        Inv_F = Inv_I - Dem_ajust

        # Si el Inventario F es negativo...
        if Inv_F < 0:
            Faltante += abs(Inv_F)  # Se agregara al contador de faltantes
            Inv_F = 0     # Por lo tanto el inventario pasara a 0
        else:
            Faltante = 0  # Resetear faltante si no hay faltante

        # Si en Inventario F es menor a 100
        if Inv_F < 100:
            Ordenes += 1
            Pedido_pendiente += 200
            t = tiempo_entrega(R, T_entrega)
            mes_entrega = mes + t
            if mes_entrega in Mes_pedido:
                Mes_pedido[mes_entrega] += 200
            else:
                Mes_pedido[mes_entrega] = 200

        if Inv_F == 0:
          IP = (Inv_I/2) * (Inv_I/Dem_ajust)
          Inv_Prom = math.ceil(IP)
        else:
          IP = (Inv_I + Inv_F)/2
          Inv_Prom = math.ceil(IP)

        print(f"Mes: {mes}, Dem_ajust: {Dem_ajust}, II: {Inv_I}, IF: {Inv_F}, Falt: {Faltante}, InvProm:{Inv_Prom}, Orden: {Ordenes}, Pedidos Pendientes: {Mes_pedido}")

        # Guardar los datos en las listas
        Mes.append(mes)
        Num_aleatorio.append(R)
        Dem_ajustada.append(Dem_ajust)
        Inv_final.append(Inv_F)
        Faltantes.append(Faltante)
        Num_orden.append(Ordenes)
        Inv_promedio.append(Inv_Prom)

        Inv_I = Inv_F - Faltante  # Ajustar Inv_I restando el faltante

    data = {
        'Mes': Mes,
        'Inv_inicial': Inv_inicial,
        'Num_aleatorio': Num_aleatorio,
        'Dem_ajustada': Dem_ajustada,
        'Inv_final': Inv_final,
        'Faltantes': Faltantes,
        'Num_orden': Num_orden,
        'Inv_promedio': Inv_promedio
    }

    df = pd.DataFrame(data)

    print(df.to_string(index=False))

In [ ]:
Inv_entario()

Mes: 1, Dem_ajust: 58, II: 150, IF: 92, Falt: 0, InvProm:121, Orden: 1, Pedidos Pendientes: {3: 200}
Mes: 2, Dem_ajust: 47, II: 92, IF: 45, Falt: 0, InvProm:69, Orden: 2, Pedidos Pendientes: {3: 200, 4: 200}
Mes: 3, Dem_ajust: 42, II: 245, IF: 203, Falt: 0, InvProm:224, Orden: 2, Pedidos Pendientes: {4: 200}
Mes: 4, Dem_ajust: 41, II: 403, IF: 362, Falt: 0, InvProm:383, Orden: 2, Pedidos Pendientes: {}
Mes: 5, Dem_ajust: 38, II: 362, IF: 324, Falt: 0, InvProm:343, Orden: 2, Pedidos Pendientes: {}
Mes: 6, Dem_ajust: 32, II: 324, IF: 292, Falt: 0, InvProm:308, Orden: 2, Pedidos Pendientes: {}
Mes: 7, Dem_ajust: 44, II: 292, IF: 248, Falt: 0, InvProm:270, Orden: 2, Pedidos Pendientes: {}
Mes: 8, Dem_ajust: 41, II: 248, IF: 207, Falt: 0, InvProm:228, Orden: 2, Pedidos Pendientes: {}
Mes: 9, Dem_ajust: 47, II: 207, IF: 160, Falt: 0, InvProm:184, Orden: 2, Pedidos Pendientes: {}
Mes: 10, Dem_ajust: 60, II: 160, IF: 100, Falt: 0, InvProm:130, Orden: 2, Pedidos Pendientes: {}
Mes: 11, Dem_ajus